In [6]:
import requests
import json
from tqdm import tqdm
import pandas as pd
import numpy as np

In [7]:
DATA_PATH = './data/'
DATA = DATA_PATH + 'BindingDB_All.tsv'

## Let's look for a suitable protein family to study

Step 1. Match targets to their families. We made a choice to use PANTHER classification, although other classifications exist too.

In [8]:
df = pd.read_csv(DATA, sep="\t", on_bad_lines='skip', dtype={
    "BindingDB Reactant_set_id": np.int32,
})

/tmp/ipykernel_615022/1259589622.py:1: DtypeWarning: Columns (8,9,10,11,12,13,15,17,18,20,21,22,23,27,28,32,33,35,36,45,46,47,48,50,51,52,53,54,55,57,58,59,60,62,63,64,65,66,67,69,70,71,72,74,75,76,77,78,79,81,82,83,86,87,88,89,90,91,93,94,95,98,99,100,101,102,103,105,106,107,110,111,112,113,114,115,117,118,119,122,123,124,125,126,127,129,130,131,134,135,136,137,138,139,146,147,148,149,150,151,158,159,160,161,162,163,170,172,173,174,175) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA, sep="\t", on_bad_lines='skip', dtype={


In [9]:
import sys
sys.path.append('./src/utils')
from retrieve_family import retrieve_family

uniprot_ids = df["UniProt (SwissProt) Entry Name of Target Chain"].unique()
# Retrieve family information for the test ID
try:
    family_dict = retrieve_family(uniprot_ids)
    # Save results to a file
    with open("protein_families.json", "w") as f:
        json.dump(family_dict, f, indent=2)
    print("Protein families saved to 'protein_families.json'")
except Exception as e:
    print(f"Error: {e}")

100%|██████████| 6533/6533 [24:02<00:00,  4.53it/s]  

Number of failed requests: 0
Protein families saved to 'protein_families.json'


In [21]:
df_family = pd.DataFrame.from_dict(family_dict, orient='index', columns=['Family'])
# Number of unique protein families
print(f"There are {df_family['Family'].nunique()} unique families in this dataset")
print(f"Number of NaNs in 'Family' column: {df_family['Family'].isna().sum()}")
# Number of targets within each family
df_family['Family'].value_counts()

There are 2118 unique families in this dataset
Number of NaNs in 'Family' column: 0


Family
NEUROTRANSMITTER GATED ION CHANNEL                                               78
ADRENERGIC RECEPTOR-RELATED G-PROTEIN COUPLED RECEPTOR                           73
5-HYDROXYTRYPTAMINE RECEPTOR                                                     67
G-PROTEIN COUPLED RECEPTOR                                                       63
-                                                                                59
                                                                                 ..
OVOCHYMASE-RELATED                                                                1
SMALL UBIQUITIN-RELATED MODIFIER                                                  1
UPF0600 PROTEIN C5ORF51                                                           1
EXTRACELLULAR GUANYL-SPECIFIC RIBONUCLEASE RNTA (AFU_ORTHOLOGUE AFUA_4G03230)     1
RPA-RELATED PROTEIN RADX                                                          1
Name: count, Length: 2118, dtype: int64

In [23]:
# Only keep families with at least n targets
n = 5
df_family = df_family.groupby('Family').filter(lambda x: len(x) >= n)
print(f"After only keeping families with at least {n} occurences, there are {df_family['Family'].nunique()} unique families in this dataset")

After only keeping families with at least 5 occurences, there are 320 unique families in this dataset


In [25]:
# In the original dataframe, only keep the rows with UniProt IDs that have a family
df_filtered = df[df['UniProt (SwissProt) Entry Name of Target Chain'].isin(df_family.index)]
# Add the family column to the original dataframe
df_filtered = df_filtered.join(df_family, on='UniProt (SwissProt) Entry Name of Target Chain')

In [29]:
# Check the amount of missing Ki
print(f"Number of NaNs in 'Ki (nM)' column: {df_filtered['Ki (nM)'].isna().sum()}")
# Check the amount of missing IC50
print(f"Number of NaNs in 'IC50 (nM)' column: {df_filtered['IC50 (nM)'].isna().sum()}")
# Drop the rows with missing IC50
df_filtered = df_filtered.dropna(subset=['IC50 (nM)'])

Number of NaNs in 'Ki (nM)' column: 1236425
Number of NaNs in 'IC50 (nM)' column: 640021


In [36]:
# drop "-" family
df_filtered = df_filtered[df_filtered['Family'] != '-']
# Group by family and count the number of rows, sort by count
df_grouped = df_filtered.groupby('Family').count().sort_values('IC50 (nM)', ascending=False)
# Count the number of unique UniProt IDs in each family
df_grouped['UniProt IDs'] = df_filtered.groupby('Family')['UniProt (SwissProt) Entry Name of Target Chain'].nunique()
df_grouped
df_grouped['Ligand to Target Ratio'] = df_grouped['IC50 (nM)'] / df_grouped['UniProt IDs']
# Sort by the ratio
df_grouped = df_grouped.sort_values('Ligand to Target Ratio', ascending=False)
df_grouped

/tmp/ipykernel_615022/1606214186.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_grouped['UniProt IDs'] = df_filtered.groupby('Family')['UniProt (SwissProt) Entry Name of Target Chain'].nunique()
/tmp/ipykernel_615022/1606214186.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_grouped['Ligand to Target Ratio'] = df_grouped['IC50 (nM)'] / df_grouped['UniProt IDs']


,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Primary ID of Target Chain.12,UniProt (SwissProt) Secondary ID(s) of Target Chain.12,UniProt (SwissProt) Alternative ID(s) of Target Chain.12,UniProt (TrEMBL) Submitted Name of Target Chain.12,UniProt (TrEMBL) Entry Name of Target Chain.12,UniProt (TrEMBL) Primary ID of Target Chain.12,UniProt (TrEMBL) Secondary ID(s) of Target Chain.12,UniProt (TrEMBL) Alternative ID(s) of Target Chain.12,UniProt IDs,Ligand to Target Ratio
Family,,,,,,,,,,,,,,,,,,,,,
PHOSPHOINOSITIDE-SPECIFIC PHOSPHOLIPASE C FAMILY PROTEIN,22248,22248,17002,17002,22248,22248,22248,20122,0,22248,...,0,0,0,0,0,0,0,0,4,5562.000000
PHOSPHATIDYLINOSITOL KINASE,44600,44600,42726,42726,44600,44600,44600,43961,19,44600,...,0,0,0,0,0,0,0,0,19,2347.368421
"RAS, DI-RAS, AND RHEB FAMILY MEMBERS OF SMALL GTPASE SUPERFAMILY",18608,18608,18087,18087,18608,18608,18608,13083,4,18608,...,0,0,0,0,0,0,0,0,8,2326.000000
AMINE OXIDASE,16289,16289,16242,16242,16289,16289,16289,2971,0,16289,...,0,0,0,0,0,0,0,0,8,2036.125000
ACETYLCHOLINESTERASE,28300,28300,28025,28025,28300,28300,28300,22579,46,28300,...,0,0,0,0,0,0,0,0,14,2021.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GUANYL-NUCLEOTIDE EXCHANGE FACTOR,3,3,3,3,3,3,3,3,0,3,...,0,0,0,0,0,0,0,0,1,3.000000
GLYCOSYLTRANSFERASE-RELATED,2,2,2,2,2,2,2,2,0,2,...,0,0,0,0,0,0,0,0,1,2.000000
TREHALASE,8,8,8,8,8,8,8,4,0,8,...,0,0,0,0,0,0,0,0,5,1.600000


In [38]:
# Save the filtered dataframe to a new file
df_grouped.to_csv(DATA_PATH + 'BindingDB_counts.tsv', sep='\t', index=False)

In [43]:
# Show proteins belonging to PHOSPHOINOSITIDE-SPECIFIC PHOSPHOLIPASE C FAMILY PROTEIN
df_filtered[df_filtered['Family'] == 'PHOSPHOINOSITIDE-SPECIFIC PHOSPHOLIPASE C FAMILY PROTEIN']['UniProt (SwissProt) Entry Name of Target Chain'].unique()

array(['BTK_HUMAN', 'PLCG1_BOVIN', 'PLCB1_HUMAN', 'PLCB_HUMAN'],
      dtype=object)

In [45]:
# Find "PLC" in UniProt IDs
df_filtered[df_filtered['UniProt (SwissProt) Entry Name of Target Chain'].str.contains('PLC', na=False)]['UniProt (SwissProt) Entry Name of Target Chain'].unique()

array(['PLCB3_HUMAN', 'PLCG1_BOVIN', 'PLCB1_HUMAN', 'PLCB_HUMAN',
       'PLC_BACCE'], dtype=object)